# Import Libraries

In [23]:
import pandas as pd

from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm




import scrub_and_explore as se
import model as m
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Obtain Data

Airbnb data for Washington DC (as of 9/22/19): http://insideairbnb.com/about.html

![image png](img/data.png)

Based on the filters used on Airbnb, we will use the following variables

- guests_included
- is_business_travel_ready
- room_type
- instant_bookable
- beds
- bedrooms
- bathrooms
- host_is_superhost
- amenities
- property_type
- neighbourhood_cleansed

In [24]:
listings = pd.read_csv('data/listings.csv.gz')

# Filter dataframe to desired columns
var_list = ['id', 'price', 'guests_included', 'is_business_travel_ready', 'room_type', 'instant_bookable', 'beds', 'bedrooms', 'bathrooms', 'host_is_superhost', 'property_type', 'neighbourhood_cleansed', 'amenities']
listings = listings.loc[:,var_list]

# Scrub and Explore Data

## Clean Data

In [25]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9189 entries, 0 to 9188
Data columns (total 13 columns):
id                          9189 non-null int64
price                       9189 non-null object
guests_included             9189 non-null int64
is_business_travel_ready    9189 non-null object
room_type                   9189 non-null object
instant_bookable            9189 non-null object
beds                        9180 non-null float64
bedrooms                    9184 non-null float64
bathrooms                   9178 non-null float64
host_is_superhost           9188 non-null object
property_type               9189 non-null object
neighbourhood_cleansed      9189 non-null object
amenities                   9189 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 933.4+ KB


Couple of things to note:
- Price is of type object, so we should convert that to numeric
- There are a few missing values. We will drop them later

### Clean price

In [26]:
listings['price'] = pd.to_numeric(listings['price'].str.replace("$","").str.replace(",",""))

### Clean Categorical Variables

In [27]:
# Create list of categorical variables
categoricalColumns = list(listings.select_dtypes(include=['object']).columns)

for col in categoricalColumns:
    print(f"{col} has {len(listings[col].unique())} values")
    print(listings[col].value_counts()[:5])
    print("\n")

is_business_travel_ready has 1 values
f    9189
Name: is_business_travel_ready, dtype: int64


room_type has 4 values
Entire home/apt    6567
Private room       2363
Shared room         207
Hotel room           52
Name: room_type, dtype: int64


instant_bookable has 2 values
f    4906
t    4283
Name: instant_bookable, dtype: int64


host_is_superhost has 3 values
f    6198
t    2990
Name: host_is_superhost, dtype: int64


property_type has 23 values
Apartment      4214
House          1918
Townhouse      1421
Condominium     765
Guest suite     504
Name: property_type, dtype: int64


neighbourhood_cleansed has 39 values
Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View    911
Union Station, Stanton Park, Kingman Park                     898
Capitol Hill, Lincoln Park                                    798
Edgewood, Bloomingdale, Truxton Circle, Eckington             717
Dupont Circle, Connecticut Avenue/K Street                    711
Name: neighbourhood_cleansed, dtype: int64


- 'is_business_travel_ready' is 1 value -> drop it since every row has same value
- 'instant_bookable' is categorical dummy -> convert to dummy
- 'host_is_superhost' is categorical dummy -> convert to dummy
- convert 'room_type' to dummy
- convert 'property_type' to dummy
- convert 'neighbourhood_cleansed' to dummy
- convert 'amenities' to dummy

In [28]:
# Make changes
listings.drop(columns=['is_business_travel_ready'], axis=1, inplace=True)

listings = se.makeDummyColumnFromCategoricalDummyColumn(listings, 'instant_bookable')
listings = se.makeDummyColumnFromCategoricalDummyColumn(listings, 'host_is_superhost')

listings = se.getDummies(listings, 'room_type')
listings = se.getDummies(listings, 'property_type')
listings = se.getDummies(listings, 'neighbourhood_cleansed')

listings = se.makeDummyColumnFromMultivalueColumn(listings, 'amenities', character_list = ["{", "}", "\""], sep=",")

Just for fun, which neighborhood was dropped when creating dummy variables?

In [29]:
neighborhoods = list(listings['neighbourhood_cleansed'].unique())
neighborhoods_dummy = [x.replace('neighbourhood_cleansed_EQ_', '') for x in listings.columns if 'neighbourhood_cleansed_EQ_' in x]

set(neighborhoods) - set(neighborhoods_dummy)

{'Brightwood Park, Crestwood, Petworth'}

Just for fun, which amenity was dropped when creating dummy variables?

In [30]:
amenities_col = listings['amenities']
character_list = ["{", "}", "\""]
for char in character_list:
    amenities_col = amenities_col.str.replace(char, "")

# Create list of unique values
amenities_list = list(set([x.strip() for l in amenities_col.str.split(",") for x in l]))
amenities_list = [str(x) for x in amenities_list if str(x) not in ['', 'nan']]

amenities_dummy = [x.replace('amenities_EQ_', '') for x in listings.columns if 'amenities_EQ_' in x]

set(amenities_list) - set(amenities_dummy)


{'24-hour check-in'}

### Drop Nulls

In [31]:
listings.dropna(inplace=True)

## Explore Data

Where are listings located?

In [32]:
se.buildMap(listings, 'price', 'count', 'Number of Listings')

How expensive are listings?

In [33]:
se.buildMap(listings, 'price', 'median', 'Median Price of Listings')

Where are the waterfront properties...near water?

In [34]:
se.buildMap(listings, 'amenities_EQ_Waterfront', 'sum', 'Number of Waterfront Properties')

How many bedrooms are available?

In [35]:
se.buildMap(listings, 'bedrooms', 'mean', 'Average Number of Bedrooms')

# Model

Can I predict the price of a listing based on its attributes?

I'll use linear regression to do so, in which my dependent (y) variable is my price and my independent (x) variables are the attributes.

## Split into Train and Test Data

In [36]:
y_data = listings[['price']]
# Deleting the original columns that were used to create dummy columns
x_data = listings.drop(columns=['id', 'price', 'room_type', 'instant_bookable', 'host_is_superhost', 'property_type', 'neighbourhood_cleansed', 'amenities'])

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, random_state=20,test_size=0.2)


## Standardize Data

I only want to scale the non-dummy columns.

In [37]:
columnsToScale = ['guests_included', 'beds', 'bedrooms', 'bathrooms']
x_train_scaled, x_test_scaled = m.scaleXData(columnsToScale, x_train, x_test)

## Model - Linear Regression

In [38]:
linReg = m.runSimpleLinearRegression(x_train_scaled, y_train)
labels = ['intercept'] + list(x_train_scaled.columns)
linReg.summary(xname=labels)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.275
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     11.50
Date:                Tue, 25 Feb 2020   Prob (F-statistic):               0.00
Time:                        08:15:59   Log-Likelihood:                -51444.
No. Observations:                7336   AIC:                         1.034e+05
Df Residuals:                    7101   BIC:                         1.050e+05
Df Model:                         234                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================================================================================
                                                                                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
intercept                                                                                                                     312.9536    166.371      1.881      0.060     -13.183     639.090
guests_included                                                                                                               -25.2078      4.031     -6.253      0.000     -33.110     -17.306
beds                                                                                                                           -0.0599      5.393     -0.011      0.991     -10.632      10.513
bedrooms                                                                                                                       92.7686      5.254     17.657      0.000      82.469     103.068
bathrooms                                                                                                                      32.4288      4.498      7.209      0.000      23.611      41.246
instant_bookable_EQ_T                                                                                                          25.8832      6.921      3.740      0.000      12.315      39.451
host_is_superhost_EQ_T                                                                                                        -24.6027      8.064     -3.051      0.002     -40.411      -8.794
room_type_EQ_Hotel room                                                                                                       -51.3179     54.642     -0.939      0.348    -158.433      55.797
room_type_EQ_Private room                                                                                                    -100.2654     10.541     -9.512      0.000    -120.930     -79.601
room_type_EQ_Shared room                                                                                                     -169.1499     25.336     -6.676      0.000    -218.815    -119.485
property_type_EQ_Apartment                                                                                                    -79.4985    162.290     -0.490      0.624    -397.635     238.638
property_type_EQ_Barn                                                                                                        -227.5642    255.534     -0.891      0.373    -728.487     273.358
property_type_EQ_Bed and breakfast                                                                                            -66.1722    165.783     -0.399      0.690    -391.156     258.812
property_ty

We have r^2 value of 0.275. Let's see if we can improve on this

## Model - Linear Regression Refined

Let's rerun the regression model, this time only using 'significant' variables from the previous regression

In [39]:
coefTable = m.runSimpleLinearRegression(x_train_scaled, y_train).summary(xname=labels).tables[1]
coefTable = coefTable.data
coefDF = m.makeCoefficientDF(coefTable)


sigVars = coefDF.loc[coefDF['P>|t|']<0.05].index
print(len(sigVars))
x_train_scaled_sigVars = x_train_scaled[sigVars]
x_test_scaled_sigVars = x_test_scaled[sigVars]

linRegRef = m.runSimpleLinearRegression(x_train_scaled_sigVars, y_train)
labels = ['intercept'] + list(x_train_scaled_sigVars.columns)

linRegRef.summary(xname=labels)

48


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     51.13
Date:                Tue, 25 Feb 2020   Prob (F-statistic):               0.00
Time:                        08:15:59   Log-Likelihood:                -51558.
No. Observations:                7336   AIC:                         1.032e+05
Df Residuals:                    7287   BIC:                         1.036e+05
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
======================================================================================================================================================================================
                                                                                                                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
intercept                                                                                                            228.7750     20.404     11.212      0.000     188.776     268.774
guests_included                                                                                                      -27.2195      3.886     -7.005      0.000     -34.837     -19.603
bedrooms                                                                                                              94.3789      4.422     21.345      0.000      85.711     103.046
bathrooms                                                                                                             34.1685      3.907      8.744      0.000      26.509      41.828
instant_bookable_EQ_T                                                                                                 23.9892      6.712      3.574      0.000      10.832      37.146
host_is_superhost_EQ_T                                                                                               -28.7879      7.579     -3.798      0.000     -43.645     -13.931
room_type_EQ_Private room                                                                                            -86.5838      9.013     -9.606      0.000    -104.252     -68.915
room_type_EQ_Shared room                                                                                            -163.8197     23.425     -6.993      0.000    -209.739    -117.901
neighbourhood_cleansed_EQ_Capitol Hill, Lincoln Park                                                                  82.9804     12.454      6.663      0.000      58.568     107.393
neighbourhood_cleansed_EQ_Cathedral Heights, McLean Gardens, Glover Park                                              61.2138     27.013      2.266      0.023       8.261     114.166
neighbourhood_cleansed_EQ_Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace    55.1170     28.546      1.931      0.054      -0.841     111.075
neighbourhood_cleansed_EQ_Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street               97.4343     17.675      5.513      0.000      62.787     132.082
neighbourhood_cleansed_EQ_Dupont Circle, Connecticut Avenue/K Street                                                  58.0803     13.222      4.393      0.000      32.161      83.999
neighbourhood_cleansed_EQ_Edgewood, Bloomingdale, Truxton Circle, Eckington                                           28.7723     12.724      2.261      0.

We now have R^2 of 0.252. Considering that there were hundreds of variables in the original model, filtering down the variable set to only a fraction of that with only a 0.02 difference in R^2 is quite an improvement.

## Model - Ridge Regression

In [40]:
alpha_array = np.array([1000, 100, 10, 1, .1, .01, .001, 0])

# From our alpha array, determine best alpha to use for ridge regression

grid_search_ridge = m.conductGridSearch(model=Ridge(), alpha_array=alpha_array, x_data=x_train_scaled, y_data=y_train)
best_alpha_ridge = grid_search_ridge.best_estimator_.alpha

C:\Users\anees\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\anees\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [41]:
# Ridge Regression
ridge = m.runRidgeRegression(x_train_scaled, y_train, best_alpha_ridge)
print(f"Ridge R^2: {ridge.score(x_train_scaled, y_train)}")
print(f"Ridge # features: {len([x for x in ridge.coef_[0] if abs(x)>10**-2])}")

Ridge R^2: 0.26430057963731823
Ridge # features: 237


## Model - Lasso Regression

In [42]:
# From our alpha array, determine best alpha to use for lasso regression

grid_search_lasso = m.conductGridSearch(model=Lasso(), alpha_array=alpha_array, x_data=x_train_scaled, y_data=y_train)
best_alpha_lasso = grid_search_lasso.best_estimator_.alpha

C:\Users\anees\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\anees\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 88974160.83493254, tolerance: 38656.84341188139
  positive)
C:\Users\anees\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4781802.993364394, tolerance: 54646.172972398286
  positive)
C:\Users\anees\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective

In [43]:
# Lasso Regression
lasso = m.runLassoRegression(x_train_scaled, y_train, best_alpha_lasso)
print(f"Lasso R^2: {lasso.score(x_train_scaled, y_train)}")
print(f"Lasso # features: {len([x for x in lasso.coef_ if abs(x)>10**-2])}")

Lasso R^2: 0.2532599313192625
Lasso # features: 60


The tests produced similar R^2. The refined Linear Regression has fewest variables, so I will use that.

# Interpret

Let's look at the variable table and see which variables have the highest effect on price

In [44]:
# Create another column for absolute value of coef
coefDF['coefABS'] = coefDF['coef'].apply(lambda x: abs(x))

In [45]:
# Top 10 variables
coefDF.sort_values(by='coefABS', ascending=False).head()

,coef,std err,t,P>|t|,[0.025,0.975],coefABS
,,,,,,,
amenities_EQ_Touchless faucets,-637.4125,938.558,-0.679,0.497,-2477.266,1202.441,637.4125
amenities_EQ_Mudroom,610.6533,1515.242,0.403,0.687,-2359.672,3580.979,610.6533
amenities_EQ_Wine cooler,514.5362,670.091,0.768,0.443,-799.042,1828.114,514.5362
amenities_EQ_Printer,-502.6934,1382.882,-0.364,0.716,-3213.554,2208.167,502.6934
amenities_EQ_High-resolution computer monitor,458.1319,805.490,0.569,0.570,-1120.868,2037.132,458.1319


In [46]:
# Bottom 10 variables
coefDF.sort_values(by='coefABS', ascending=False).tail()

,coef,std err,t,P>|t|,[0.025,0.975],coefABS
,,,,,,,
amenities_EQ_Full kitchen,-1.269800e+00,3.546400e+01,-0.036,0.971,-7.078900e+01,6.825000e+01,1.269800e+00
"neighbourhood_cleansed_EQ_Deanwood, Burrville, Grant Park, Lincoln Heights, Fairmont Heights",8.249000e-01,4.802300e+01,0.017,0.986,-9.331500e+01,9.496400e+01,8.249000e-01
beds,-5.990000e-02,5.393000e+00,-0.011,0.991,-1.063200e+01,1.051300e+01,5.990000e-02
amenities_EQ_Exercise equipment,-6.719000e-12,1.600000e-11,-0.420,0.674,-3.810000e-11,2.460000e-11,6.719000e-12
amenities_EQ_Double oven,4.029000e-13,4.460000e-12,0.090,0.928,-8.340000e-12,9.150000e-12,4.029000e-13


In [47]:
# Rerun Linear Regression using sci-kit learn
lm = LinearRegression()
lm.fit(x_train_scaled_sigVars, y_train)
y_train_pred = lm.predict(x_train_scaled_sigVars)
r2_score(y_train, y_train_pred)

0.251927331132777

In [48]:
# Applying model to test data
y_test_pred = lm.predict(x_test_scaled_sigVars)
r2_score(y_test, y_test_pred)

0.2808436392800463

Our test dataset produced a r^2 value of 0.28, an increase from our training r^2 score. A couple of things to note:
- the data doesn't factor in occupancy rate. So an owner can put a listing on the site, but it's unclear if that listing has been reserved at that price
- since amenities are self-reported, reporting certain amenities are voluntary. So if a listing lacks a certain amenitity, it MIGHT be the case that the owner didn't feel the need to list an amenitiy
- there's the possibility that more amenities could relate to a decrease in price. The owner may not feel confident about the listing and may try to compensate by listing a lot more amenities than usual, and it may be unclear if listings have listed amenities based on the data alone.

These reasons could explain while r^2 has been consistently low.